In [1]:
import pandas as pd
import nltk
from nltk.corpus import state_union
from nltk.tokenize import PunktSentenceTokenizer
import spacy

from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()


df = pd.read_csv('datasets/wiki_movie_plots_deduped.csv')

df.head()

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot
0,1901,Kansas Saloon Smashers,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...,"A bartender is working at a saloon, serving dr..."
1,1901,Love by the Light of the Moon,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"The moon, painted with a smiling face hangs ov..."
2,1901,The Martyred Presidents,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"The film, just over a minute long, is composed..."
3,1901,"Terrible Teddy, the Grizzly King",American,Unknown,NaN,unknown,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...",Lasting just 61 seconds and consisting of two ...
4,1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",NaN,unknown,https://en.wikipedia.org/wiki/Jack_and_the_Bea...,The earliest known adaptation of the classic f...


In [2]:
df_two_cols = df[['Title','Plot']]


In [5]:
import pprint
pp = pprint.PrettyPrinter(indent=4)

cnt_entities = {}
total = len(df)


for i in range(10):
    doc = nlp(df['Plot'][i])
    current_entities = [(X.text, X.label_) for X in doc.ents]
    for entity in current_entities:
        print('current entity', entity)
        cnt_entities[entity[0]] = entity
# for plot in df['Plot']:
#     doc = nlp(plot)
#     current_entities = [(X.text, X.label_) for X in doc.ents]
    
#     # iterate over the entities, and add it to the dictionary
#     for entity in current_entities:
#         cnt_entities[entity[0]] = entity

cnt_entities


current entity ('Irish', 'NORP')
current entity ('Carrie Nation', 'ORG')
current entity ('Irish', 'NORP')
current entity ('Nation', 'ORG')
current entity ('night', 'TIME')
current entity ('two', 'CARDINAL')
current entity ('first', 'ORDINAL')
current entity ('three', 'CARDINAL')
current entity ('U.S.', 'GPE')
current entity ('Abraham Lincoln', 'PERSON')
current entity ('James A. Garfield', 'PERSON')
current entity ('William McKinley', 'PERSON')
current entity ('second', 'ORDINAL')
current entity ('just over eight seconds', 'TIME')
current entity ('Lady Justice', 'ORG')
current entity ('just 61 seconds', 'TIME')
current entity ('two', 'CARDINAL')
current entity ('first', 'ORDINAL')
current entity ('winter', 'DATE')
current entity ('Theodore Roosevelt', 'PERSON')
current entity ('Two', 'CARDINAL')
current entity ('His Photographer', 'WORK_OF_ART')
current entity ('second', 'ORDINAL')
current entity ('Jack', 'PERSON')
current entity ('Jack', 'PERSON')
current entity ('Jack', 'PERSON')
cur

{'Irish': ('Irish', 'NORP'),
 'Carrie Nation': ('Carrie Nation', 'ORG'),
 'Nation': ('Nation', 'ORG'),
 'night': ('night', 'TIME'),
 'two': ('two', 'CARDINAL'),
 'first': ('first', 'ORDINAL'),
 'three': ('three', 'CARDINAL'),
 'U.S.': ('U.S.', 'GPE'),
 'Abraham Lincoln': ('Abraham Lincoln', 'PERSON'),
 'James A. Garfield': ('James A. Garfield', 'PERSON'),
 'William McKinley': ('William McKinley', 'PERSON'),
 'second': ('second', 'ORDINAL'),
 'just over eight seconds': ('just over eight seconds', 'TIME'),
 'Lady Justice': ('Lady Justice', 'ORG'),
 'just 61 seconds': ('just 61 seconds', 'TIME'),
 'winter': ('winter', 'DATE'),
 'Theodore Roosevelt': ('Theodore Roosevelt', 'PERSON'),
 'Two': ('Two', 'CARDINAL'),
 'His Photographer': ('His Photographer', 'WORK_OF_ART'),
 'Jack': ('Jack', 'PERSON'),
 'the "Garden"': ('the "Garden"', 'FAC'),
 'a "Dog': ('a "Dog', 'WORK_OF_ART'),
 'the "White Rabbit\'s': ('the "White Rabbit\'s', 'FAC'),
 'House': ('House', 'ORG'),
 "The Duchess's Cheshire Cat"

In [ ]:
import re

words = re.findall(r'\w+', text.lower())
cnt_words = Counter(words)

In [ ]:
doc = nlp(cnt_words.most_common(10)[0][0])

words = " ".join(plot for plot in df['Plot'])

doc = nlp('New York City')
pp.pprint([(X.text, X.label_) for X in doc.ents])